# 🎨 NeMo Data Designer: Generate Diverse RAG Evaluations

> ⚠️ **Warning**: NeMo Data Designer is current in Early Release and is not recommended for production use.
>
> **Note**: In order to run this notebook, you must have the NeMo Data Designer microservice deployed locally via docker compose. See the [deployment guide](http://docs.nvidia.com/nemo/microservices/latest/set-up/deploy-as-microservices/data-designer/docker-compose.html) for more details.
>
> Alternatively, you can use the [NeMo Data Designer managed service](https://build.nvidia.com/nemo/data-designer). Please refer the [intro-tutorials](../../intro-tutorials/1-the-basics.ipynb) on how to connect to it. 
>
> **Note**: If you are using the NeMo Data Designer managed service, you will only be able to launch preview jobs. You will not be able to launch jobs using the `create` method.

This tutorial demonstrates how to generate comprehensive evaluation datasets for Retrieval-Augmented Generation (RAG) systems, customized to your content and use cases. 

You'll learn how to create diverse question-answer pairs at scale, covering a variety of difficulty levels and reasoning types, including both answerable and unanswerable scenarios.



#### 💾 Install dependencies

**IMPORTANT** 👉 If you haven't already, follow the instructions in the [README](../../README.md) to install the necessary dependencies. Note you may need to restart your kernel after setting up the environment.


In [ ]:
from nemo_microservices import NeMoMicroservices
from nemo_microservices.beta.data_designer import (
    DataDesignerConfigBuilder,
    DataDesignerClient,
)
from nemo_microservices.beta.data_designer.config import columns as C
from nemo_microservices.beta.data_designer.config import params as P

### ⚙️ Initialize the NeMo Data Designer Client

- The data designer client is responsible for submitting generation requests to the Data Designer microservice.
- In this notebook, we connect to a local deployment of data designer. You can deploy your own instance of data designer by following the deployment instructions [here](https://docs.nvidia.com/nemo/microservices/latest/set-up/deploy-as-microservices/data-designer/docker-compose.html).


In [ ]:
data_designer_client = DataDesignerClient(client=NeMoMicroservices(base_url="http://localhost:8080"))

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- You must provide a list of model configs to the builder at initialization.

- This list contains the models you can choose from (via the `model_alias` argument) during the generation process.


In [ ]:
# We specify the endpoint of the model during deployment using the model_provider_registry.
model_id = "/raid/models/nemotron-nano-9b-v2"
# model_id = "nvidia/nvidia-nemotron-nano-9b-v2"
model_alias = "nemotron-nano-9b-v2"

In [ ]:
config_builder = DataDesignerConfigBuilder(
    model_configs=[
        P.ModelConfig(
            alias=model_alias,
            provider="local-llm",
            model=model_id,
            inference_parameters=P.InferenceParameters(
                max_tokens=1024,
                temperature=0.6,
                top_p=0.95,
            ),
            is_reasoner=True
        ),
    ]
)

### 📕 Source Document Configuration

Let's define our source documents and the total number of evaluation pairs we want to generate. You can replace the document list with your own PDFs, web pages, or other text sources.

In [ ]:
# Define source documents and total number of evaluation pairs to generate
# You can replace this with your own documents
DOCUMENT_LIST = ["./data/databricks-state-of-data-ai-report.pdf"]

### ⚙️ Document Processing

Now we'll create a Document Processor class that handles loading and chunking the source documents. 

This class uses langchain's RecursiveCharacterTextSplitter and unstructured.io for robust document parsing.

In [ ]:
from typing import List, Union
from langchain.text_splitter import RecursiveCharacterTextSplitter
from unstructured.partition.auto import partition
import tempfile
import os

In [ ]:
class DocumentProcessor:
    """Handles loading and chunking source documents for RAG evaluation."""

    def __init__(self, chunk_size: int = 4192, chunk_overlap: int = 200):
        """Initialize with configurable chunk size and overlap."""
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

    def parse_document(self, uri: str) -> str:
        """Parse a single document from URI into raw text."""
        with open(uri, 'rb') as file:
            content = file.read()
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(content)
                temp_file.flush()
                elements = partition(temp_file.name)

        os.unlink(temp_file.name)
        return "\n\n".join([str(element) for element in elements])

    def process_documents(self, uris: Union[str, List[str]]) -> List[str]:
        """Process one or more documents into chunks for RAG evaluation."""
        if isinstance(uris, str):
            uris = [uris]

        all_chunks = []
        for uri in uris:
            text = self.parse_document(uri)
            chunks = self.text_splitter.split_text(text)
            all_chunks.extend(chunks)

        return all_chunks

### Data Models

Let's define Pydantic models for structured output generation. These schemas will ensure our generated data has consistent structure and validation.

In [ ]:
from pydantic import BaseModel, Field

class QAPair(BaseModel):
    question: str = Field(
        ..., description="A specific question related to the domain of the context"
    )
    answer: str = Field(
        ..., description="Either a context-supported answer or explanation of why the question cannot be answered"
    )
    reasoning: str = Field(
        ..., description="A clear and traceable explanation of the reasoning behind the answer"
    )

### Processing Documents and Setting Up Data Designer

Now we'll process our document chunks and set up the Data Designer with our seed dataset.

**Note**: At this time, we only support using a single file as the seed. If you have multiple files you would like to use as seeds, it is recommended you consolidated these into a single file. 

In [ ]:
import pandas as pd

# Process document chunks
processor = DocumentProcessor(chunk_size=4192, chunk_overlap=200)
chunks = processor.process_documents(DOCUMENT_LIST)

# Create a seed DataFrame with the document chunks
seed_df = pd.DataFrame({"context": chunks})
seed_df.head()

In [ ]:
os.makedirs("data", exist_ok=True)
seed_df.to_csv("data/document_chunks.csv", index=False)


In [ ]:
# Upload the seed dataset with document chunks
# Using shuffle with replacement allows the model to reuse context chunks
config_builder.with_seed_dataset(
    repo_id="advanced-tutorials/rag_evaluation_dataset",
    filename="document_chunks.csv",
    dataset_path="./data/document_chunks.csv",
    sampling_strategy="shuffle",
    with_replacement=True,
    datastore={"endpoint": "http://localhost:3000/v1/hf"},
)

### Adding Categorical Columns for Controlled Diversity

Now we'll add categorical columns to control the diversity of our RAG evaluation pairs. We'll define:

1. **Difficulty levels**: easy, medium, hard

2. **Reasoning types**: factual recall, inferential reasoning, etc.

3. **Question types**: answerable vs. unanswerable (with weighting)

In [ ]:
# Configure categorical columns for controlled diversity
config_builder.add_column(
    C.SamplerColumn(
        name="difficulty",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["easy", "medium", "hard"],
            description="The difficulty level of the question"
        )
    )
)

config_builder.add_column(
    C.SamplerColumn(
        name="reasoning_type",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=[
                "factual recall",
                "inferential reasoning",
                "comparative analysis",
                "procedural understanding",
                "cause and effect"
            ],
            description="The type of reasoning required to answer the question"
        )
    )
)

config_builder.add_column(
    C.SamplerColumn(
        name="question_type",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["answerable", "unanswerable"],
            # 10:1 ratio of answerable to unanswerable questions.
            weights=[10, 1],
        )
    )
)

config_builder.validate()

### Adding LLM-Structured Column for Q&A Pair Generation

Now let's set up the core of our data generation: the Q&A pair column that will produce structured question-answer pairs based on our document context and control parameters.

In [ ]:
# Add Q&A pair generation column
config_builder.add_column(
    C.LLMStructuredColumn(
        name="qa_pair",
        model_alias=model_alias,
        system_prompt=(
            "You are an expert at generating high-quality RAG evaluation pairs. "
            "You are very careful in assessing whether the question can be answered from the provided context. "
        ),
        prompt="""\
{{context}}

Generate a {{difficulty}} {{reasoning_type}} question-answer pair.
The question should be {{question_type}} using the provided context.

For answerable questions:
- Ensure the answer is fully supported by the context

For unanswerable questions:
- Keep the question topically relevant
- Make it clearly beyond the context's scope
""",
        output_format=QAPair
    )
)

config_builder.validate()

### Adding Evaluation Metrics with Custom Rubrics

To assess the quality of our generated Q&A pairs, we'll add evaluation metrics using detailed rubrics for scoring. 

We use Data Designer's `LLMJudgeColumn` for this, defining a set of custom Rubrics designed for our task.

In [ ]:
context_relevance_rubric = P.Rubric(
    name="Context Relevance",
    description="Evaluates how relevant the answer is to the provided context",
    scoring={
        "5": "Perfect relevance to context with no extraneous information",
        "4": "Highly relevant with minor deviations from context",
        "3": "Moderately relevant but includes some unrelated information",
        "2": "Minimally relevant with significant departure from context",
        "1": "Almost entirely irrelevant to the provided context"
    }
)

answer_precision_rubric = P.Rubric(
    name="Answer Precision",
    description="Evaluates the accuracy and specificity of the answer",
    scoring={
        "5": "Extremely precise with exact, specific information",
        "4": "Very precise with minor imprecisions",
        "3": "Adequately precise but could be more specific",
        "2": "Imprecise with vague or ambiguous information",
        "1": "Completely imprecise or inaccurate"
    }
)

answer_completeness_rubric = P.Rubric(
    name="Answer Completeness",
    description="Evaluates how thoroughly the answer addresses all aspects of the question",
    scoring={
        "5": "Fully complete, addressing all aspects of the question",
        "4": "Mostly complete with minor omissions",
        "3": "Adequately complete but missing some details",
        "2": "Substantially incomplete, missing important aspects",
        "1": "Severely incomplete, barely addresses the question"
    }
)

hallucination_avoidance_rubric = P.Rubric(
    name="Hallucination Avoidance",
    description="Evaluates the absence of made-up or incorrect information",
    scoring={
        "5": "No hallucinations, all information is factual and verifiable",
        "4": "Minimal hallucinations that don't impact the core answer",
        "3": "Some hallucinations that partially affect the answer quality",
        "2": "Significant hallucinations that undermine the answer",
        "1": "Severe hallucinations making the answer entirely unreliable"
    }
)

EVAL_METRICS_PROMPT_TEMPLATE = """\
You are an expert evaluator of question-answer pairs. Analyze the following Q&A pair and evaluate it objectively.

For this {{difficulty}} {{reasoning_type}} Q&A pair:
{{qa_pair}}

Take a deep breath and carefully evaluate each criterion based on the provided rubrics, considering the difficulty level and reasoning type indicated.
"""

config_builder.add_column(
    C.LLMJudgeColumn(
        name="eval_metrics",
        model_alias=model_alias,
        prompt=EVAL_METRICS_PROMPT_TEMPLATE,
        rubrics=[context_relevance_rubric, answer_precision_rubric, answer_completeness_rubric, hallucination_avoidance_rubric],
    )
)

config_builder.validate()

### 👀 Preview Sample Records

Let's generate a preview to see what our data will look like before running the full generation.

In [ ]:
preview = data_designer_client.preview(config_builder, verbose_logging=True)

In [ ]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

In [ ]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset.head()

### Generate the Full Dataset

Now let's generate our full dataset of RAG evaluation pairs, analyze the coverage, and export it to a JSONL file for use in evaluating RAG systems.

In [ ]:
# Generate the full dataset.
job_results = data_designer_client.create(config_builder, num_records=20, wait_until_done=False)

job_results.wait_until_done()

In [ ]:
dataset = job_results.load_dataset()
print("\nGenerated dataset shape:", dataset.shape)

In [ ]:
# Export the dataset to JSONL format.
dataset.to_json('./data/rag_evals.jsonl', orient='records', lines=True)
print("\nDataset exported to ./data/rag_evals.jsonl")

### Using Your RAG Evaluation Dataset

Now that you've generated a diverse RAG evaluation dataset, here are some ways to use it:

1. **Benchmarking**: Test your RAG system against these evaluation pairs to measure performance

2. **Error Analysis**: Identify patterns in where your RAG system struggles

3. **Optimization**: Use insights to tune retrieval and generation parameters

4. **Regression Testing**: Track performance over time as you improve your system

5. **Model Comparison**: Compare different LLMs, retrievers, or RAG architectures

The JSONL file contains structured data with questions, ground truth answers, and quality metrics that you can use with most evaluation frameworks.